In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

 Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

 target_trandform usage??

In [2]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

26422272it [01:24, 312549.97it/s]                               


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



29696it [00:00, 184897.05it/s]                          


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



4422656it [00:04, 935591.47it/s]                              


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



6144it [00:00, 6141516.63it/s]          

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
batch_size = 64

# DataLoader supports automatic batching, sampling, shuffling and multiprocess data loading
training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Model definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(             # sequential object that passes the input through the cascaded objects given in the argument list
            nn.Linear(in_features=28*28, out_features=512), # returns a linear layer object
            nn.ReLU(),                                      
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=10)
        )

    def forward(self, x):
        x = self.flatten(x)         # converts the matrix image into a vector. !!2d relations disappear when we do this though
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Search
1. CrossEntropyLoss

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [37]:
def train(dataloader, model, loss_fn, optimizer):
    global print_pred
    size = len(dataloader.dataset)
    model.train() # This will tell the model that you are training the model. So effectively layers like dropout, batchnorm etc. which behave different on the train and test procedures know what is going on and hence can behave accordingly
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [40]:

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()    # similar to what was said in the train function
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
--------------------
loss: 2.312057 [    0/60000]
loss: 2.294950 [ 6400/60000]
loss: 2.278666 [12800/60000]
loss: 2.272128 [19200/60000]
loss: 2.247820 [25600/60000]
loss: 2.225778 [32000/60000]
loss: 2.237456 [38400/60000]
loss: 2.205414 [44800/60000]
loss: 2.202818 [51200/60000]
loss: 2.175300 [57600/60000]
Test Error: 
 Accuracy: 37.1%, Avg loss: 2.166854

Epoch 2
--------------------
loss: 2.182889 [    0/60000]
loss: 2.173635 [ 6400/60000]
loss: 2.122420 [12800/60000]
loss: 2.139332 [19200/60000]
loss: 2.079340 [25600/60000]
loss: 2.023248 [32000/60000]
loss: 2.060291 [38400/60000]
loss: 1.985484 [44800/60000]
loss: 1.987467 [51200/60000]
loss: 1.927067 [57600/60000]
Test Error: 
 Accuracy: 49.4%, Avg loss: 1.917714

Epoch 3
--------------------
loss: 1.950247 [    0/60000]
loss: 1.925639 [ 6400/60000]
loss: 1.817788 [12800/60000]
loss: 1.861013 [19200/60000]
loss: 1.736455 [25600/60000]
loss: 1.685800 [32000/60000]
loss: 1.717689 [38400/60000]
loss: 1.617144 [44800/60000]

In [18]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [39]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(pred)
    predicted, actual = classes[pred[0].argmax()], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

tensor([[-2.0795, -2.3361, -0.9355, -1.8216, -0.8981,  2.1887, -1.1087,  2.5210,
          1.6062,  2.8885]])
Predicted: 'Ankle boot', Actual: 'Ankle boot'
